In [1]:
import numpy as np
import pandas as pd 

dataset = pd.read_csv('../dataset/dados_pregao_intb3.csv')
colunas = ['preco_ultimo_negocio']
data = dataset[colunas]
train_size = int(len(data) * 0.8)
train_data, test_data = data[:train_size], data[train_size:]

train_data = train_data.values
test_data = test_data.values

data

,preco_ultimo_negocio
0,28.30
1,28.04
2,27.38
3,27.41
4,27.60
...,...
888,29.54
889,28.91
890,29.14
891,28.69


In [2]:
def create_sequences(data, sequence_length):
    X, y = [], []
    for i in range(len(data) - sequence_length):
        X.append(data[i:i+sequence_length])
        y.append(data[i+sequence_length])
    return np.array(X), np.array(y)

sequence_length = 10
X_train, y_train = create_sequences(train_data, sequence_length)
X_test, y_test = create_sequences(test_data, sequence_length)

In [3]:
from keras.models import Sequential, save_model, load_model
from keras.layers import LSTM, Dense
from keras.callbacks import EarlyStopping
from keras.losses import MeanSquaredError
from keras.metrics import RootMeanSquaredError
from keras.optimizers import Adam

model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(sequence_length, 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')

model.fit(X_train, y_train, epochs=100, batch_size=32)

predictions = model.predict(X_test)

mse = np.mean((predictions - y_test) ** 2)
print("Mean Squared Error:", mse)

Epoch 1/100


e:\Workspace\pregao_forecasting\.venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 268.7364
Epoch 2/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 5.5294
Epoch 3/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 4.0543
Epoch 4/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3.3497
Epoch 5/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3.4919
Epoch 6/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3.4079
Epoch 7/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3.3511
Epoch 8/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.7553
Epoch 9/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.6158
Epoch 10/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3.1439
Epoch 11/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3.1413
Epoch 12/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.7304
Epoch 13/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.9161
Epoch 14/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.9553
Epoch 15/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3.7800
Epoch 16/100
2

In [4]:
test_predictions1 = model.predict(X_test).flatten()

X_test_list = []
for i in range(len(X_test)):
    X_test_list.append(X_test[i][0])
    

test_predictions_df1 = pd.DataFrame({'X_test':list(X_test_list), 
                                    'LSTM Prediction':list(test_predictions1)})

test_predictions_df1

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


,X_test,LSTM Prediction
0,[23.62],23.948444
1,[24.08],23.548626
2,[23.64],23.600180
3,[23.58],23.600971
4,[24.1],23.286572
...,...,...
164,[28.55],29.286827
165,[29.1],29.554550
166,[24.23],30.037308
167,[28.26],29.811296
